In [26]:
import pyFAI
import numpy
import fabio
import collections
import numba

In [4]:
class MarchingSquareMPL(object):

    def __init__(self, image, mask=None):
        import matplotlib._cntr
        self._image = image
        self._mask = mask
        x, y = numpy.mgrid[:self._image.shape[0], :self._image.shape[1]]
        self._contour = matplotlib._cntr.Cntr(x, y, self._image)

    _deltas = [(0.0, 0.0), (0.99, 0.0), (0.0, 0.99), (0.99, 0.99)]

    def _flag_coord_over_mask(self, coord):
        """Flag coord over the mask as NaN"""
        for dx, dy in self._deltas:
            if self._mask[int(coord[0] + dx), int(coord[1] + dy)] != 0:
                return float("nan"), float("nan")
        return coord

    def iso_contour(self, value):
        res = self._contour.trace(value)
        nseg = len(res) // 2
        polylines, _codes = res[:nseg], res[nseg:]

        result = []
        for polyline in polylines:
            if self._mask is not None:
                polyline = map(self._flag_coord_over_mask, polyline)
                polyline = list(polyline)
                polyline = numpy.array(polyline)
            # swap x and y columns
            polyline = numpy.array([polyline[:,1], polyline[:,0]]).T
            result.append(polyline)
        return result

In [58]:
class MarchingSquarePyFai(object):

    def __init__(self, image, mask=None):
        import pyFAI.ext.marchingsquares
        self._image = image
        self._mask = mask

    def iso_contour(self, value):
        import pyFAI.ext.marchingsquares
        points = pyFAI.ext.marchingsquares.isocontour(self._image, value, sorted=False).round().astype(int)
        if self._mask is not None:
            # points = numpy.array([[i[1], i[0]] for i in points if not mask[i[1], i[0]]])
            points = points[numpy.logical_not(self._mask[points.T[1], points.T[0]])]
        if len(points) == 0:
            return []
        points = numpy.unique(points, axis=0)
        polylines = self._extract_polylines(points)
        return polylines

    @staticmethod
    def _is_same(p1, p2):
        return p1[0] == p2[0] and p1[1] == p2[1]

    @staticmethod
    def _is_segment(p1, p2):
        dx, dy = abs(p1[0] - p2[0]), abs(p1[1] - p2[1])
        return dx <= 1 and dy <= 1

    @staticmethod
    def _dist(p1, p2):
        dx, dy = abs(p1[0] - p2[0]), abs(p1[1] - p2[1])
        return dx + dy

    @classmethod
    def _extract_polylines(cls, points):
        """Incremental algorythm to extract a set of polylines from an unsorted
        set of coordinates.

        A segment have defined as 2 point closer than 2
        """
        polys = []

        for p in points:
            found = []
            for poly_id, poly in enumerate(polys):
                begin, end, polyline = poly
                if cls._is_same(p, begin) or cls._is_same(p, end):
                    assert(False)
                elif cls._is_segment(p, begin):
                    description = (poly_id, 0)
                    if cls._dist(p, begin) == 1:
                        found.insert(0, description)
                    else:
                        found.append(description)
                elif cls._is_segment(p, end):
                    description = (poly_id, 1)
                    if cls._dist(p, end) == 1:
                        found.insert(0, description)
                    else:
                        found.append(description)

            if len(found) == 0:
                # That's a new polygon
                begin, end, polyline = p, p, [p]
                polys.append([begin, end, polyline])
            elif len(found) == 1:
                # Incrase a polygon
                poly_id, loc = found[0]
                poly = polys[poly_id]
                if loc == 0:
                    # incrate at begining
                    poly[0] = p
                    poly[2].insert(0, p)
                elif loc == 1:
                    # incrate at end
                    poly[1] = p
                    poly[2].append(p)
                else:
                    assert(False)
            elif len(found) >= 2:
                # merge 2 polygones
                poly1_id, loc1 = found[0]
                poly2_id, loc2 = found[1]
                poly1 = polys[poly1_id][2]
                poly2 = polys[poly2_id][2]

                if loc1 == 0:
                    poly1.reverse()
                elif loc1 == 1:
                    pass
                else:
                    assert(False)

                if loc2 == 0:
                    pass
                elif loc2 == 1:
                    poly2.reverse()
                else:
                    assert(False)

                if poly2_id > poly1_id:
                    polys.pop(poly2_id)
                    polys.pop(poly1_id)
                else:
                    polys.pop(poly1_id)
                    polys.pop(poly2_id)

                polyline = []
                polyline.extend(poly1)
                polyline.append(p)
                polyline.extend(poly2)
                begin, end = polyline[0], polyline[-1]
                polys.append([begin, end, polyline])
            else:
                assert(False)

        result = []
        for poly in polys:
            result.append(numpy.array(poly[2], dtype=numpy.int))

        return result

In [64]:
class MarchingSquarePyFaiNumba(object):

    def __init__(self, image, mask=None):
        import pyFAI.ext.marchingsquares
        self._image = image
        self._mask = mask

    def iso_contour(self, value):
        import pyFAI.ext.marchingsquares
        points = pyFAI.ext.marchingsquares.isocontour(self._image, value, sorted=False).round().astype(int)
        if self._mask is not None:
            # points = numpy.array([[i[1], i[0]] for i in points if not mask[i[1], i[0]]])
            points = points[numpy.logical_not(self._mask[points.T[1], points.T[0]])]
        if len(points) == 0:
            return []
        points = numpy.unique(points, axis=0)
        polylines = _extract_polylines(points)
        return polylines

@numba.jit
def _dist2(p1, p2):
    dx, dy = abs(p1[0] - p2[0]), abs(p1[1] - p2[1])
    return dx * dx + dy * dy

@numba.jit
def _extract_polylines(points):
    """Incremental algorythm to extract a set of polylines from an unsorted
    set of coordinates.

    A segment have defined as 2 point closer than 2
    """
    polys = []

    for p in points:
        found = []
        for poly_id, poly in enumerate(polys):
            begin, end, polyline = poly
            d1 = _dist2(p, begin)
            d2 = _dist2(p, end)
            if d1 == 0 or d2 == 0:
                assert(False)
            elif d1 <= 2:
                description = (poly_id, 0)
                if d1 <= 1:
                    found.insert(0, description)
                else:
                    found.append(description)
            elif d2 <= 2:
                description = (poly_id, 1)
                if d2 <= 1:
                    found.insert(0, description)
                else:
                    found.append(description)

        if len(found) == 0:
            # That's a new polygon
            begin, end, polyline = p, p, [p]
            polys.append([begin, end, polyline])
        elif len(found) == 1:
            # Incrase a polygon
            poly_id, loc = found[0]
            poly = polys[poly_id]
            if loc == 0:
                # incrate at begining
                poly[0] = p
                poly[2].insert(0, p)
            elif loc == 1:
                # incrate at end
                poly[1] = p
                poly[2].append(p)
            else:
                assert(False)
        elif len(found) >= 2:
            # merge 2 polygones
            poly1_id, loc1 = found[0]
            poly2_id, loc2 = found[1]
            poly1 = polys[poly1_id][2]
            poly2 = polys[poly2_id][2]

            if loc1 == 0:
                poly1.reverse()
            elif loc1 == 1:
                pass
            else:
                assert(False)

            if loc2 == 0:
                pass
            elif loc2 == 1:
                poly2.reverse()
            else:
                assert(False)

            if poly2_id > poly1_id:
                polys.pop(poly2_id)
                polys.pop(poly1_id)
            else:
                polys.pop(poly1_id)
                polys.pop(poly2_id)

            polyline = []
            polyline.extend(poly1)
            polyline.append(p)
            polyline.extend(poly2)
            begin, end = polyline[0], polyline[-1]
            polys.append([begin, end, polyline])
        else:
            assert(False)

    result = []
    for poly in polys:
        result.append(numpy.array(poly[2], dtype=numpy.int))

    return result

In [38]:
Problem = collections.namedtuple("Problem", ["image", "mask", "values"])

In [44]:
def create_problem1():
    ROOT = "/workspace/valls/pyfai.git/_own/iso"
    data = fabio.open(ROOT + "/data.tif").data
    mask = fabio.open(ROOT + "/mask.tif").data
    mask = mask != 0
    values = range(10, 1000, int(240/6))[0:7]
    return Problem(data, mask, values)

problem1 = create_problem1()

In [45]:
marching_square = MarchingSquareMPL(problem1.image, problem1.mask)
%timeit marching_square.iso_contour(140)

10 loops, best of 3: 39.9 ms per loop


In [46]:
marching_square = MarchingSquarePyFai(problem1.image, problem1.mask)
%timeit marching_square.iso_contour(140)

10 loops, best of 3: 45.3 ms per loop


In [65]:
marching_square = MarchingSquarePyFaiNumba(problem1.image, problem1.mask)
%timeit marching_square.iso_contour(140)

The slowest run took 44.32 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 26.8 ms per loop


In [50]:
def create_problem2():
    ROOT = "/workspace/valls/pyfai.git/_own/iso"
    data = numpy.load(ROOT + "/wos_tth.npz")
    image = data["tth"]
    mask = fabio.open(ROOT + "/wos_mask.edf").data
    mask = mask != 0
    values = data["angles"]
    return Problem(image, mask, values)

problem2 = create_problem2()

In [60]:
marching_square = MarchingSquareMPL(problem2.image, problem2.mask)
%timeit [marching_square.iso_contour(angle) for angle in problem2.values]

1 loop, best of 3: 377 ms per loop


In [62]:
marching_square = MarchingSquarePyFai(problem2.image, problem2.mask)
%timeit [marching_square.iso_contour(a) for a in problem2.values]

1 loop, best of 3: 849 ms per loop


In [66]:
marching_square = MarchingSquarePyFaiNumba(problem2.image, problem2.mask)
%timeit [marching_square.iso_contour(a) for a in problem2.values]

1 loop, best of 3: 441 ms per loop
